In [ ]:
!pip install datasets
!pip install peft

## 데이터 전처리

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd "/content/drive/MyDrive/2024_2학기 학교 데이터/Intro_to_AI_data"

raw_data = pd.read_csv('csrc_illegalSite_data_성균관대.csv')
raw_data = raw_data[['keyword', 'label']]
raw_data

In [ ]:
raw_data.isnull().sum()

### 레이블 개수 확인

In [ ]:
label_num = len(raw_data["label"].unique())
print(f'라벨 개수: {label_num}')
print(f'데이터 라벨 종류: {raw_data["label"].unique()}')

print(f'각 라벨 개수: {raw_data["label"].value_counts()}')

In [ ]:
raw_data['label'] = raw_data['label'] - 1

label_to_site_dict = {0: "일반사이트", 1: "도박사이트", 2: "도박 제외 불법사이트"}

print(f'각 라벨 개수: {raw_data["label"].value_counts()}')

In [ ]:
import matplotlib.pyplot as plt

str_length = [len(untokenized_data) for untokenized_data in raw_data['keyword']]

plt.hist(str_length, bins = 100)
plt.title('Length of Untokenized Data')
plt.show()

In [ ]:
max(str_length)

In [ ]:
print(raw_data.shape)  # 전체 데이터의 크기 확인
print(raw_data['keyword'].isnull().sum())  # text 열에 결측값이 있는지 확인
print(raw_data['label'].isnull().sum())  # label 열에 결측값이 있는지 확인

In [ ]:
print(raw_data['keyword'].shape)
print(raw_data['label'].shape)

### 데이터로더 정의

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer



# 데이터셋을 train/validation 세트로 나누기 (80%:20% 비율)

# 허깅페이스 Dataset 포맷으로 변환
dataset = Dataset.from_pandas(raw_data)


# 전처리 함수 정의: Tokenization
def preprocess_function(data):
  return tokenizer(data['keyword'], truncation = True, padding = 'max_length', max_length = 512, return_tensors = 'pt')

# one_hot encoding으로 변환
def label2one_hot(data):
  label = data['label']
  one_hot = [0] * label_num
  one_hot[label] = 1
  data['label'] = one_hot
  return data


checkpoint = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


tokenized_data = dataset.map(preprocess_function, batched = True)
#tokenized_data = tokenized_data.map(label2one_hot)

# 'label' 컬럼을 'labels'로 변경 (허깅페이스)
tokenized_data = tokenized_data.rename_column('label', "labels")
tokenized_data = tokenized_data.rename_column('keyword', "text")

# 데이터셋을 Trainer API로 사용하기 위해 필요한 포맷으로 설정
tokenized_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(tokenized_data)

In [ ]:
split_data = tokenized_data.train_test_split(test_size = 0.1)
split_data

In [ ]:
# 데이터 확인
sample = split_data['train'][0]['input_ids']
sample_encoded = tokenizer.decode(sample, skip_special_tokens = True, clean_up_tokenization_spaces = True)

print(tokenized_data, "\n")
print(sample)
print(sample_encoded)

In [ ]:
# tokenizer가 자동으로 맨 앞에 [CLS] 토큰 넣어줌

tokenizer.cls_token_id

## 모델 아키텍쳐

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
checkpoint = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

### 허깅페이스 Token 설정

In [ ]:
"""

from huggingface_hub import login, Repository

# hugging-face 토큰
# 절대 유출 금지
access_token = ''
login(access_token)

# 허깅페이스 허브 연결
#repo_name = 'iontail/TwoStageDistilBERT_Gambling'
#repo = Repository(local_dir = 'TwoStageDistilBERT_Gambling', clone_from = repo_name)

checkpoint_interval = 1 # 매 epoch마다 체크포인트 저장
"""

In [ ]:
# 모델 버전 확인
model_ver = 3

save_model_path = "/content/drive/MyDrive/2024_2학기 학교 데이터/Intro_to_AI_data/TwoStageDistilBERT_LoRA_ver" + str(model_ver) + '.pt'

save_hist_path = "/content/drive/MyDrive/2024_2학기 학교 데이터/Intro_to_AI_data/hist_ver" + str(model_ver) + '.pt'

#체크포인트 저장 함수
def save_checkpoint(model, optimizer, scheduler, epoch, loss, save_path):
  checkpoint = {
      "model_state_dict": model.state_dict(),
      "optimizer_state_dict": optimizer.state_dict(),
      "scheduler_state_dict": scheduler.state_dict() if scheduler else None,
      "epoch": epoch,
      "loss": loss
  }

  torch.save(checkpoint, save_path)



def save_hist(train_loss, train_acc, val_loss, val_acc, train_f1, val_f1, save_path):
  hist_dict = {
      "train_loss_list": train_loss,
      "train_acc_list": train_acc,
      "val_loss_list": val_loss,
      "val_acc_list": val_acc,
      "train_f1_list": train_f1,
      "val_f1_list": val_f1
  }

  torch.save(hist_dict, save_path)


def load_checkpoint(model, optimizer, scheduler, file_path, hist_path):
  checkpoint = torch.load(file_path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  if scheduler:
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']

  hist_checkpoint = torch.load(hist_path)
  train_loss =  hist_checkpoint["train_loss_list"]
  train_acc = hist_checkpoint["train_acc_list"]
  val_loss = hist_checkpoint["val_loss_list"]
  val_acc = hist_checkpoint["val_acc_list"]
  train_f1 = hist_checkpoint("train_f1_list")
  val_f1 = hist_checkpoint("val_f1_list")


  print(f"Checkpoint loaded from {file_path}, starting from epoch {epoch+1}")
  return model, optimizer, scheduler, epoch + 1, loss, train_loss, train_acc, val_loss, val_acc, train_f1, val_f1  # 불러올 때 다음 에포크부터 시작

In [ ]:
#DistilBERT로 모델 불러오면 labels를 입력으로 받지 않기 때문에 DistilBertForSequenceClassification 사용

from transformers import DistilBertForSequenceClassification, get_scheduler
from peft import get_peft_model, LoraConfig, TaskType


class TwoStageDistilBERT_LoRA(nn.Module):
  def __init__(self, distilbert_checkpoint, num_labels_1stage = 2, num_labels_2stage = 3):
    super(TwoStageDistilBERT_LoRA, self).__init__()


    # 첫 번째 stage
    self.distilbert1 = DistilBertForSequenceClassification.from_pretrained(distilbert_checkpoint,
                                                                           num_labels = num_labels_1stage, ignore_mismatched_sizes = True,
                                                                           output_hidden_states=True)

    lora_config1 = LoraConfig(task_type = TaskType.SEQ_CLS, r = 8, lora_alpha = 32, target_modules = ['q_lin', 'v_lin'], lora_dropout = 0.1 )
    self.distilbert1 = get_peft_model(self.distilbert1, lora_config1)

    # 두 번째 stage
    self.distilbert2 = DistilBertForSequenceClassification.from_pretrained(distilbert_checkpoint,
                                                                           num_labels = num_labels_2stage, ignore_mismatched_sizes = True)

    lora_config2 = LoraConfig(task_type = TaskType.SEQ_CLS, r = 8, lora_alpha = 32, target_modules = ['q_lin', 'v_lin'], lora_dropout = 0.1 )
    self.distilbert2 = get_peft_model(self.distilbert2, lora_config2)


  def forward(self, input_ids,  attention_mask, labels1 = None, labels2 = None):
    output1 = self.distilbert1(input_ids = input_ids, attention_mask = attention_mask, labels = labels1)
    hidden1 = output1.hidden_states[-1] # 마지막 레이어의 hidden state 가져오기
    logits1 = output1.logits

    pred1 = torch.argmax(logits1, dim = 1)

    output2 = self.distilbert2(inputs_embeds = hidden1, attention_mask = attention_mask, labels = labels2)
    logits2 = output2.logits

    total_loss = output1.loss + output2.loss


    return total_loss, logits1, logits2

  """
  #체크포인트 설정하여 저장 및 불러오기 위한 함수들 정의
  def save_pretrained(self, save_directory):
    os.makedirs(save_directory, exist_ok = True)

    self.distilbert1.save_pretrained(os.path.join(save_directory, "stage1"))
    torch.save(self.distilbert1.state_dict(), os.path.join(save_directory, "stage1", "pytorch_model.bin"))

    self.distilbert2.save_pretrained(os.path.join(save_directory, "stage2"))
    torch.save(self.distilbert2.state_dict(), os.path.join(save_directory, "stage2", "pytorch_model.bin"))


  @classmethod
  def from_pretrained(cls, save_directory):
      # 저장된 모델 불러오기
      model = cls.__new__(cls)

      model.distilbert1.load_state_dict(torch.load(os.path.join(save_directory, "stage1", "pytorch_model.bin")))
      model.distilbert2.load_state_dict(torch.load(os.path.join(save_directory, "stage2", "pytorch_model.bin")))
      return model


  def get_hist(self):
    pass
  """


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

batch_size = 16

train_DL = DataLoader(split_data['train'], shuffle = True, batch_size = batch_size)
val_DL = DataLoader(split_data['test'], shuffle = True, batch_size = batch_size)

In [ ]:
#####################
"""
모델 중단되었으면 아래 변수를 'True'로 설정
"""
resume_training = False

# warm-up step 확인
# pre-trained 모델은 2%인데 너무 적어서 5%로 변경
import math

EPOCH = 20

total_steps = EPOCH * len(train_DL)
warm_up = int(total_steps * 0.05)

learning_rate = 1e-5

LAMBDA = 0.01

model = TwoStageDistilBERT_LoRA(distilbert_checkpoint = checkpoint).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), weight_decay = LAMBDA, lr = learning_rate)
scheduler = get_scheduler('linear', optimizer = optimizer, num_warmup_steps = warm_up, num_training_steps = total_steps)



if resume_training:
  model, optimizer, scheduler, start_epoch, last_loss, train_loss, train_acc, val_loss, val_acc, train_f1, val_f1 = load_checkpoint(model, optimizer, scheduler, save_model_path, save_hist_path)

#model.to(DEVICE)
model

In [ ]:
def output2label(logits1, logits2, label_to_site_dict):
  N = logits1.shape[0]
  logits1 = np.array(logits1.cpu())
  logits2 = np.array(logits2.cpu())



  output1 = np.argmax(logits1, axis = 1)
  #print(output1, "\n",[label_to_site_dict[out] for out in output1])

  mask = (output1 != 0)
  output2 = np.argmax(logits2[mask], axis = 1)
  #print(output2, "\n", [label_to_site_dict[out] for out in output2])

  result = []
  result_label = []
  idx = 0
  for out in output1:
    if out == 0:
      result_label.append(out)
      result.append(label_to_site_dict[out])
    else:
      result_label.append(output2[idx])
      result.append(label_to_site_dict[output2[idx]])
      idx += 1


  print(result)
  print(f"Result: {result_label}")

def output2num(logits1, logits2):
  N = logits1.shape[0]

  logits1 = logits1.detach().cpu().numpy()
  logits2 = logits2.detach().cpu().numpy()

  output1 = np.argmax(logits1, axis = 1)
  mask = (output1 != 0)
  output2 = np.argmax(logits2[mask], axis = 1)

  result = np.array([])
  idx = 0
  for out in output1:
    if out == 0:
      result = np.append(result, out)
    else:
      result = np.append(result, output2[idx])
      idx += 1

  return result



def get_metrics(logits1, logits2, labels2):

  result = output2num(logits1, logits2)
  labels2 = np.array(labels2.cpu())

  correct = (result == labels2).sum().item()

  TP_mask = (result == 1)
  TP = (labels2[TP_mask] == 1).sum().item()

  FP = (labels2[TP_mask] != 1).sum().item()

  FN_mask = (result != 1)
  FN = (labels2[FN_mask] == 1).sum().item()

  TN = (labels2[FN_mask] != 1).sum().item()

  ## correct ##

  """
  # 예측값을 얻기 위해 logits에서 가장 높은 값을 선택
  predictions1 = torch.argmax(logits1, dim = 1)
  predictions2 = torch.argmax(logits2, dim = 1)

  mask = (predictions1 == (labels1 == 0)) # 정상사이트는 정상사이트로 정상 분류했을 때 해당 자리는 1로 set
  reverse_mask = (mask != 1) # 유해사이트로 분류시 해당 인덱스는 1로 set

  correct1 = (mask == 1).sum().item()

  # 첫 번째 Stage에서 유해사이트로 분류되었을 때,  2nd Stage에서의 분류 결과
  correct2 = (predictions2[reverse_mask] == labels2[reverse_mask])
  correct2 = correct2.sum().item()
  acc = (correct1 + correct2)
  """

  # 레이블이 3개여서 correct != TP + TN
  return correct, TP, FP, FN, TN


In [ ]:
# 모델 테스트

model.eval()

with torch.no_grad():
  for batch in train_DL:
    batch = {key: value.to(DEVICE) for key, value in batch.items()}

    labels1 = batch['labels']  # 첫 번째 분류용 라벨
    #labels1 = torch.where(torch.argmax(labels1, dim = 1) == 0, 0, 1) # Binary label 처리 (0이면 0, 나머지 1, 2이면 1)
    labels1 = torch.where(labels1 ==0, 0, 1)
    labels2 = batch['labels']


    loss, logits1, logits2 = model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'], labels1=labels1, labels2=labels2)
    print(loss)
    print(logits1)
    print(labels2)

    output2label(logits1, logits2, label_to_site_dict)
    print(f"Label: {labels2}")
    print(get_metrics(logits1, logits2, labels2))
    print("-"*60, "\n")

    #acc = get_acc(logits1, logits2, labels1, labels2)
    #print(f"Accuracy: {acc / batch_size}")


    break

### 추론 테스트

In [ ]:
model.eval()

# 입력 텍스트
input_text = "뉴스 국회 최근 합법화 국회의원 국민 헌법 법 제정"
# 텍스트를 토큰화
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# 입력에 대한 추론 (추론에서는 gradient 필요없음)

model.eval()

with torch.no_grad():
    input_ids = inputs['input_ids'].to(DEVICE)
    attention_mask = inputs['attention_mask'].to(DEVICE)

    # 1단계 모델에 입력
    output1 = model.distilbert1(input_ids=input_ids, attention_mask=attention_mask)
    logits1 = output1.logits

    hidden1 = output1.hidden_states[-1]  # 마지막 레이어의 hidden state
    output2 = model.distilbert2(inputs_embeds=hidden1, attention_mask=attention_mask)
    logits2 = output2.logits
    output2label(logits1, logits2, label_to_site_dict)


### 학습 파라미터 지정

### 파인튜닝 시작

In [ ]:
# Early Stopping 커스텀 클래스 선언
class EarlyStopping:
  def __init__(self, patience = 3, min_delta = 0):
    """
    Args:
        patience: 몇 번 연속으로 성능향상이 없을 때 종료할 것인지
        min_delta: 성능 향상의 최소치
    """
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.best_loss = None
    self.early_stop = False

  def __call__(self, val_loss):
    if self.best_loss is None:
      self.best_loss = val_loss
    elif val_loss < (self.best_loss - self.min_delta):
      self.best_loss = val_loss
      self.counter = 0
    else:
      self.counter +=1
      if self.counter >= self.patience:
        self.early_stop = True


In [ ]:
def fine_tuning(model, train_DL, val_DL, resume_training):

  early_stopping = EarlyStopping(patience = 3)

  if resume_training:
    start = start_epoch
    total_train_loss = train_loss
    total_train_acc =  train_acc
    total_val_loss = val_loss
    total_val_acc = val_acc
    total_train_f1 = train_f1
    total_val_f1 = val_f1

  else:
    start = 0
    total_train_loss = []
    total_train_acc =  []
    total_val_loss = []
    total_val_acc = []
    total_train_f1 = []
    total_val_f1 = []

  N = len(val_DL)
  for epoch in range(start, EPOCH):
    model.train()
    total_loss = 0
    total_acc = 0
    val_loss_batch = 0
    val_acc_batch = 0

    TP_batch = 0
    FP_batch = 0
    FN_batch = 0

    TP_batch_val = 0
    FP_batch_val = 0
    FN_batch_val = 0

    tqdm_batch = tqdm(train_DL, desc = f"Epoch {epoch + 1} / {EPOCH}")
    i = 1

    for batch in tqdm_batch:
      # acc 구하는데 사용 / interation


      batch = {key: value.to(DEVICE) for key, value in batch.items()}
      labels1 = batch['labels']  # 첫 번째 분류용 라벨
      labels1 = torch.where(labels1 ==0, 0, 1)

      labels2 = batch['labels']  # 두 번째 분류용 라벨

      optimizer.zero_grad()
      loss, logits1, logits2 = model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'], labels1=labels1, labels2=labels2)
      loss.backward()
      optimizer.step()
      scheduler.step()

      total_loss += loss.item() * batch_size
      tqdm_batch.set_postfix(loss = total_loss / i)
      i += 1

      batch_acc, TP, FP, FN, _ = get_metrics(logits1, logits2, labels2)
      total_acc += batch_acc
      TP_batch += TP
      FP_batch += FP
      FN_batch += FN


    # 한 에폭마다 훈련 결과 저장
    total_train_loss.append(total_loss/ len(train_DL))
    total_train_acc.append(total_acc / (len(train_DL) * batch_size))
    total_train_f1.append((2 * TP_batch) / (2 * TP_batch + FP_batch + FN_batch))


    # Validation
    model.eval()
    tqdm_batch_val = tqdm(val_DL, desc = f"Validation")
    j = 1

    for batch in tqdm_batch_val:


      batch = {key: value.to(DEVICE) for key, value in batch.items()}
      labels1 = batch['labels']  # 첫 번째 분류용 라벨
      labels1 = torch.where(labels1 ==0, 0, 1)

      labels2 = batch['labels']  # 두 번째 분류용 라벨

      loss, logits1, logits2 = model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'], labels1=labels1, labels2=labels2)

      val_loss_batch += loss.item() * batch_size
      tqdm_batch_val.set_postfix(loss = val_loss_batch / j)
      j +=1

      batch_acc, TP, FP, FN, _ = get_metrics(logits1, logits2, labels2)
      val_acc_batch += batch_acc
      TP_batch_val += TP
      FP_batch_val += FP
      FN_batch_val += FN

    total_val_loss.append(val_loss_batch / len(val_DL))
    total_val_acc.append(val_acc_batch / (len(val_DL) * batch_size))
    total_val_f1.append((2 * TP_batch_val) / (2 * TP_batch_val + FP_batch_val + FN_batch_val))

    #조기종료
    early_stopping(val_loss_batch / N)

    if early_stopping.early_stop:
      print("Early Stopping is Triggered!!!!")
      break
    print(f'Accuracy: {val_acc_batch / (N * batch_size)}%')
    save_checkpoint(model, optimizer, scheduler, epoch, loss, save_model_path)
    save_hist(total_train_loss, total_train_acc, total_val_loss, total_val_acc, total_train_f1, total_val_f1, save_hist_path)



In [ ]:
fine_tuning(model, train_DL, val_DL, resume_training)

### 훈련결과 시각화

In [ ]:
hist_data = torch.load(save_hist_path)

train_loss =  hist_data["train_loss_list"]
train_acc = hist_data["train_acc_list"]
val_loss = hist_data["val_loss_list"]
val_acc = hist_data["val_acc_list"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 첫 번째 그래프: Train Loss
ax1.plot(range(1, len(train_loss) + 1), train_loss, label='Train', color='r')
ax1.plot(range(1, len(val_loss) + 1), val_loss, label='Val', color='b')
ax1.set_title('Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()

# 두 번째 그래프: Validation Loss
ax2.plot(range(1, len(train_acc) + 1), train_acc, label='Train', color='r')
ax2.plot(range(1, len(val_acc) + 1), val_acc, label='Val', color='b')
ax2.set_title('Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()


# 레이아웃 자동 조정
plt.tight_layout()

# 그래프 출력
plt.show()

In [ ]:
train_f1 = hist_data["train_f1_list"]
val_f1 = hist_data["val_f1_list"]

plt.plot(range(1, len(train_f1) + 1), train_f1, label = 'Train', color = 'r')
plt.plot(range(1, len(val_f1) + 1), val_f1, label = 'Val', color = 'b')
plt.title("F1-Score")
plt.xlabel('Epochs')
plt.ylabel('F1-Score')
plt.legend()

plt.show()

In [ ]:
print(f"Validation Accuracy: {val_acc[-1]:.3%}")
print(f"F1-Score: {val_f1[-1]:.3%}")

In [ ]:
print(f"Validation Accuracy: {max(val_acc):.3%}")
print(f"F1-Score: {max(val_f1):.3%}")

In [ ]:
print(val_acc)

In [ ]:
print(int(33484/(209 + 555 + 485)))

In [ ]:
print(100 * 9546 / 9786, "%")